In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import h5py
from math import pi
from scipy.stats import zscore

from analysis.config import birds, h5_path_dict, pickle_dir
from analysis.ExpData import ExpData
from analysis.utils import get_fr

# Collect population patterns and save into pickle file
Visit patterns are (visits, neurs, frames) size

### Parameters

In [3]:
window = int(20*2)
fr_smoothing_window = int(20*0.25)
l2_normalization = True
pfile_name = "population_patterns_checks.p"

### Data formatting

In [ ]:
data = {}
for key in h5_path_dict.keys():
    for fil in range(len(h5_path_dict[key])):
        fil_string = h5_path_dict[key][fil].as_posix()
        print(fil_string)
        f = h5py.File(fil_string, 'r')
        exp_data = ExpData(f)
        data[fil_string] = {}
        c_visits, r_visits, ch_visits, noncrch_visits = exp_data.get_crch_visits()
        import pdb; pdb.set_trace()
        hopcentered_visits = exp_data.get_hopcentered_visits(window)
        fr = get_fr(exp_data.spikes, fr_smoothing_window)
        if l2_normalization:
            scaling = np.square(np.linalg.norm(fr, axis=1))
            #scaling = np.linalg.norm(fr, axis=1)
            fr /= scaling[:, None]
            fr[np.isnan(fr)] = 0
        visit_patterns = np.zeros(
            (hopcentered_visits.shape[0], exp_data.num_neurs, window*2 + 1)
            )
        for i, visit in enumerate(hopcentered_visits):
            visit_pattern = fr[:, visit]
            visit_pattern[:, visit == -1] = np.nan
            visit_patterns[i] = visit_pattern            
        data[fil_string]['c_visits'] = c_visits
        data[fil_string]['r_visits'] = r_visits
        data[fil_string]['ch_visits'] = ch_visits
        data[fil_string]['noncrch_visits'] = noncrch_visits
        data[fil_string]['fr'] = fr
        data[fil_string]['visit_patterns'] = visit_patterns
        data[fil_string]['visit_times'] = hopcentered_visits
with open(pickle_dir / pfile_name, 'wb') as f: 
    pickle.dump(data, f)